In [1]:
import os

# Find the latest versions of
#   Spark & Hadoop:  https://spark.apache.org/downloads.html (https://www.apache.org/dist/spark/)
#   Postgres driver: https://jdbc.postgresql.org/
os.environ['HADOOP_VERSION']   = hadoop_version   = 'hadoop3'
os.environ['SPARK_VERSION']    = spark_version    = 'spark-3.3.1'
os.environ['POSTGRES_VERSION'] = postgres_version = 'postgresql-42.5.1'

# Install Java
! apt install openjdk-11-jdk-headless > /dev/null
os.environ['JAVA_HOME']  = '/usr/lib/jvm/java-11-openjdk-amd64'

# Install Spark
! wget https://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
! tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'
! pip install findspark

# Install Postgres driver
! wget https://jdbc.postgresql.org/download/$POSTGRES_VERSION.jar

# Install AWS's Boto3
! pip install boto3

import boto3
import findspark
findspark.init()
from   getpass     import getpass
from   pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName('M17-Amazon-Challenge') \
  .config('spark.driver.extraClassPath', f'/content/{postgres_version}.jar') \
  .getOrCreate()
spark



--2022-12-18 18:11:16--  https://www.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving www.apache.org (www.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to www.apache.org (www.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz [following]
--2022-12-18 18:11:17--  https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz.1’

spark-3.3.1-bin-had 100%[===================>] 285.48M  28.2MB/s    in 11s     

2022-12-18 18:11:28 (26.4 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz.1’ saved [2993

### Load Amazon Data into Spark DataFrame

In [2]:
from pyspark import SparkFiles
url = "https://temp-haerman-dec-15.s3.ap-northeast-1.amazonaws.com/amazon_reviews_us_Baby_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...|2015-08-31 00:00:00|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...| 

In [3]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

1752932
1752727
1752727


In [4]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import col,to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(['customer_id','product_id','product_title','review_id','product_parent','review_date','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
review_df.show()

+-----------+----------+--------------------+--------------+--------------+-------------------+-----------+-------------+-----------+----+-----------------+
|customer_id|product_id|       product_title|     review_id|product_parent|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+----------+--------------------+--------------+--------------+-------------------+-----------+-------------+-----------+----+-----------------+
|   45651772|B00I39194Q|Whimsical Alphabe...| RYUQ18NWF4XAS|     182134733|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
|   39031684|B00JU949AE|Baby Einstein Act...|R147C9YMU55OPJ|      79050378|2015-08-31 00:00:00|          4|            0|          0|   N|                Y|
|    6829510|B0116UHU50|LOVE MY - 100% Pr...|R22E3HPL3OVOYU|      23805879|2015-08-31 00:00:00|          1|            0|          0|   N|                Y|
|   41429833|B00AQYZCXK|BubbleBum Travel ...|R2TTMSR2QGW7R

In [6]:
# Create the customers_table DataFrame
customers_df = review_df.groupby('customer_id').agg({'customer_id':'count'}).withColumnRenamed('count(customer_id)', 'customer_count')
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15450825|             3|
|   16014891|             1|
|     544564|             4|
|   52620152|             3|
|   18663556|             1|
|   11875435|             2|
|   14066928|             6|
|   22762916|             9|
|   26900509|             2|
|   22412167|            13|
|   48407518|             1|
|   40969364|             1|
|   51544976|            14|
|   39008558|             1|
|   14692917|             1|
|   48909019|             1|
|   13744004|             2|
|   18971789|             2|
|    3273846|             1|
|   48693125|             2|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = review_df.select(['product_id','product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
|B00WKIR22E|Poly Comfortable ...|
|B001EU35BQ|Peg Perego Tatami...|
|B001TWI81G|Thermos Reusable ...|
|B007EJ7J4U|Kaloo Doudou Beig...|
|B00M8MGGQA|Large Baby Bag Or...|
|B00Y2CJWKQ|Clinical Digital ...|
|B00Y8ION80|★ Baby Pacifier &...|
|B00UJOWZ96|Corner Cushion Ba...|
|B00DZ3K64Y|Sassy Soft Grip N...|
|B00JBYPKE2|Maxi Cosi Pria 85...|
|B00HK6GWTU|Ju-Ju-Be Be Light...|
|B00IYU4O8W|   Lamaze Cloth Book|
|B00SCDM4FQ|Olababy Silicone ...|
|B004UOTWJ2|Charlie Banana - ...|
|B001LF3YQK|Skip Hop Baby Swi...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = review_df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RYUQ18NWF4XAS|   45651772|B00I39194Q|     182134733| 2015-08-31|
|R147C9YMU55OPJ|   39031684|B00JU949AE|      79050378| 2015-08-31|
|R22E3HPL3OVOYU|    6829510|B0116UHU50|      23805879| 2015-08-31|
|R2TTMSR2QGW7RQ|   41429833|B00AQYZCXK|      77521874| 2015-08-31|
|R1OK3IRLYYW2O5|     902287|B00RGFIYSC|      76959588| 2015-08-31|
|R15DDV0SR7BN3M|   12612039|B000U5LXSS|     810367090| 2015-08-31|
|R3HEYGFQIWWQZT|    5237138|B004HKIIFS|     116994869| 2015-08-31|
|R3KCZP5ZHHHQ32|   24755112|B000XPB6UU|     235663202| 2015-08-31|
|R2ITPE0FXNH1R0|    5006626|B006SFUD4E|     491843093| 2015-08-31|
| RJWQE3MMHHA6R|   38901787|B00DYOQLNO|     357525815| 2015-08-31|
|R3O80QVLMMQ86X|   16229505|B00DJPK8MS|     292396087| 2015-08-31|
| RRE7SRK9FUSD0|   12625899|B00PHQCUA0|     281420532| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RYUQ18NWF4XAS|          5|            0|          0|   N|                Y|
|R147C9YMU55OPJ|          4|            0|          0|   N|                Y|
|R22E3HPL3OVOYU|          1|            0|          0|   N|                Y|
|R2TTMSR2QGW7RQ|          4|            1|          1|   N|                Y|
|R1OK3IRLYYW2O5|          5|            0|          0|   N|                Y|
|R15DDV0SR7BN3M|          5|            0|          0|   N|                Y|
|R3HEYGFQIWWQZT|          1|            0|          0|   N|                Y|
|R3KCZP5ZHHHQ32|          5|            0|          0|   N|                Y|
|R2ITPE0FXNH1R0|          5|            0|          0|   N|                Y|
| RJWQE3MMHHA6R|          4|            0|          0|   Y|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cu1mghc9ghoy.ap-northeast-1.rds.amazonaws.com/reviews_db"
config = {"user":"postgres", 
          "password": " ", 
          "driver":"org.postgresql.Driver"}


In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [16]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)